Import dependencies and print Tensorflow version:

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.4.0-dev20200730'

In [2]:
INPUT_SIZE = 299
INPUT_CHANNELS_COUNT = 3
CLASSES_COUNT = 10
DATASET_LENGTH = 100
BATCH_SIZE = 32

def get_empty_image(i):
    image = tf.zeros([INPUT_SIZE, INPUT_SIZE, INPUT_CHANNELS_COUNT])
    one_hot_label = np.zeros([CLASSES_COUNT], np.bool)
    one_hot_label[0] = 1
    return image, one_hot_label

dataset = tf.data.Dataset.range(DATASET_LENGTH).map(get_empty_image).batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 299, 299, 3), (None, 10)), types: (tf.float32, tf.bool)>

Load the Saved Model, disable training of its first layer (which is the Xception model) and print its summary to verify, that it is similar to the original model:

In [3]:
SAVED_MODEL_PATH = "saved_model"
saved_model = tf.keras.models.load_model(SAVED_MODEL_PATH)
saved_model.layers[0].trainable = False
saved_model.summary(line_length=120)

Model: "sequential"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
xception (Functional)                                 (None, 10, 10, 2048)                            20861480          
________________________________________________________________________________________________________________________
global_average_pooling2d (GlobalAveragePooling2D)     (None, 2048)                                    0                 
________________________________________________________________________________________________________________________
dense (Dense)                                         (None, 10)                                      20490             
Total params: 20,881,970
Trainable params: 20,490
Non-trainable params: 20,861,480
__________________________________________________

Now when trying to fit the Saved Model, we will get ResourceExhaustedError (out of memory):

In [4]:
saved_model.fit(dataset, epochs=1)

ResourceExhaustedError:  OOM when allocating tensor with shape[32,728,19,19] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/xception/block11_sepconv1/separable_conv2d (defined at <ipython-input-4-b4c664a47161>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_39217]

Function call stack:
train_function
